In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pickle
import gc


In [2]:
import sys
sys.path.insert(0, '../')

%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [3]:
from config.constants import *
from viz.plot import *
from features.statistical_features import*

In [4]:
filtered_csv_path =  f"{PROCESSED_DATA_DIR}/filtered_df_probabilities.csv"
filtered_df = pd.read_csv(filtered_csv_path)
print(filtered_df.shape)
filtered_df.head()

(28387, 16)


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,level_1
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,1.0,0.0000,0.000000,0.000,0.000000,0.000000,0
1,1628180742,6,30.0,353733,6,30.0,364593930,42516,Seizure,1.0,0.0000,0.000000,0.000,0.000000,0.000000,6
2,2277392603,0,0.0,924234,0,0.0,1978807404,30539,GPD,0.0,0.0000,0.454545,0.000,0.090909,0.454545,9
3,722738444,0,0.0,999431,0,0.0,557980729,56885,LRDA,0.0,0.0625,0.000000,0.875,0.000000,0.062500,11
4,387987538,0,0.0,1084844,0,0.0,4099147263,4264,LRDA,0.0,0.0000,0.000000,1.000,0.000000,0.000000,22


In [5]:
filtered_csv_path_2 =  f"{PROCESSED_DATA_DIR}/train_non_overlap_2.csv"
filtered_df_2 = pd.read_csv(filtered_csv_path_2)
print(filtered_df_2.shape)
print(filtered_df_2['target'].value_counts())
filtered_df_2.head()


(17089, 12)
target
Other      7196
Seizure    2785
LPD        2577
GRDA       1828
GPD        1787
LRDA        916
Name: count, dtype: int64


,eeg_id,spec_id,offset,patient_id,target,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,kl
0,568657,789577333,0.0,20654,Other,0.0,0.000000,0.25,0.000000,0.166667,0.583333,4.584192
1,582999,1552638400,0.0,20230,LPD,0.0,0.857143,0.00,0.071429,0.000000,0.071429,4.870032
2,642382,14960202,1008.0,5955,Other,0.0,0.000000,0.00,0.000000,0.000000,1.000000,7.802344
3,751790,618728447,908.0,38549,GPD,0.0,0.000000,1.00,0.000000,0.000000,0.000000,7.802344
4,778705,52296320,0.0,40955,Other,0.0,0.000000,0.00,0.000000,0.000000,1.000000,7.802344


In [6]:
train_df = pd.read_csv(f"{PROCESSED_DATA_DIR}/train_df.csv")
train_df.head()

,eeg_id,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,568657,789577333,0.0,16.0,20654,0.0,0.000000,0.25,0.000000,0.166667,0.583333,Other
1,582999,1552638400,0.0,38.0,20230,0.0,0.857143,0.00,0.071429,0.000000,0.071429,LPD
2,642382,14960202,1008.0,1032.0,5955,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other
3,751790,618728447,908.0,908.0,38549,0.0,0.000000,1.00,0.000000,0.000000,0.000000,GPD
4,778705,52296320,0.0,0.0,40955,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other


In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17089 entries, 0 to 17088
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   eeg_id        17089 non-null  int64  
 1   spec_id       17089 non-null  int64  
 2   min           17089 non-null  float64
 3   max           17089 non-null  float64
 4   patient_id    17089 non-null  int64  
 5   seizure_vote  17089 non-null  float64
 6   lpd_vote      17089 non-null  float64
 7   gpd_vote      17089 non-null  float64
 8   lrda_vote     17089 non-null  float64
 9   grda_vote     17089 non-null  float64
 10  other_vote    17089 non-null  float64
 11  target        17089 non-null  object 
dtypes: float64(8), int64(3), object(1)
memory usage: 1.6+ MB


In [10]:
original_df = pd.read_csv(f"{RAW_DATA_DIR }/train.csv")
TARGET_COLS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
original_df['sum_column'] = original_df[TARGET_COLS].sum(axis=1)
original_df.head()
original_df["sum_column"].describe()

count    106800.000000
mean          7.255496
std           5.645681
min           1.000000
25%           3.000000
50%           3.000000
75%          13.000000
max          28.000000
Name: sum_column, dtype: float64

1. For a givin df, open and group by eeg_id 
2. open the parquet file path ( f"{RAW_DATA_DIR}/train_eegs/{eeg_id}.parquet" ) based on eeg_id 
3. for each eeg_label_offset_seconds ( varible off_set ) in the group of the eeg_id, grab that window eeg_50s = eeg.iloc[eeg_offset*200:(eeg_offset+50)*200]
4. for each eeg_50s, we have colomns of [ Fp1	F3	C3	P3	F7	T3	T5	O1	Fz	Cz	Pz	Fp2	F4	C4	P4	F8	T4	T6	O2	EKG], we need to do the montage (keep EKG to be the same) and have the new colomns of [Fp1-F7   F7-T3   T3-T5   T5-O1  Fp2-F8  F8-T4  T4-T6   T6-O2   Fp1-F3   F3-C3   C3-P3   P3-O1   Fp2-F4  F4-C4   C4-P4   P4-O2   Fz-Cz   Cz-Pz   EKG] 
5. export

In [12]:
unique_eeg_ids = set(original_df['eeg_id'].unique())
folder_path = os.path.join(RAW_DATA_DIR, "train_eegs") # 请替换为你的文件夹路径
files_in_folder = [f.split('.')[0] for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
files_in_folder_ids = set(int(f) for f in files_in_folder)
extra_files = files_in_folder_ids - unique_eeg_ids

print(f"文件夹中共有 {len(extra_files)} 个未被录入的文件。")
print("未被录入的文件ID:", extra_files)

文件夹中共有 211 个未被录入的文件。
未被录入的文件ID: {3576774144, 2734605314, 3425324038, 2989528583, 3624732686, 2021270543, 1980230677, 1006575126, 2585735707, 463419419, 2437996061, 1218899486, 2883273759, 1046303268, 3374558757, 710494757, 124842023, 507760170, 876702254, 1362609712, 2564933177, 2069539903, 1484488768, 2512462400, 2370617920, 1124971078, 735478859, 397155403, 2495888973, 176751182, 4265462352, 2256830545, 512986705, 495719000, 3973402202, 535701085, 1540422245, 2163060838, 3726556263, 2704546408, 2514244198, 2518637163, 3713571437, 4118489199, 1593654895, 3965080689, 3184041077, 358214261, 888227967, 2032669824, 3861560461, 3534133394, 4066543763, 3613550741, 990852758, 2945876125, 771666080, 3029056160, 2360465057, 789759143, 595031723, 3897137837, 1844412082, 3231425203, 1114646196, 438882483, 166924982, 217188535, 1967500472, 2167910584, 3932079802, 3296877755, 2031389372, 4263916735, 1138516159, 2890567875, 1527948998, 3287073479, 1395846857, 2915092170, 2229664974, 2995737301, 380

In [5]:
def montage_transform_banana(eeg_50s):
    montage_pairs = {
        'Fp1-F7': ('Fp1', 'F7'),
        'F7-T3': ('F7', 'T3'),
        'T3-T5': ('T3', 'T5'),
        'T5-O1': ('T5', 'O1'),
        'Fp2-F8': ('Fp2', 'F8'),
        'F8-T4': ('F8', 'T4'),
        'T4-T6': ('T4', 'T6'),
        'T6-O2': ('T6', 'O2'),
        'Fp1-F3': ('Fp1', 'F3'),
        'F3-C3': ('F3', 'C3'),
        'C3-P3': ('C3', 'P3'),
        'P3-O1': ('P3', 'O1'),
        'Fp2-F4': ('Fp2', 'F4'),
        'F4-C4': ('F4', 'C4'),
        'C4-P4': ('C4', 'P4'),
        'P4-O2': ('P4', 'O2'),
        'Fz-Cz': ('Fz', 'Cz'),
        'Cz-Pz': ('Cz', 'Pz'),
        'EKG': ('EKG',)  
    }
    
    transformed_eeg = pd.DataFrame()
    
    for new_col, electrodes in montage_pairs.items():
        if len(electrodes) == 1: 
            transformed_eeg[new_col] = eeg_50s[electrodes[0]]
        else:
            transformed_eeg[new_col] = eeg_50s[electrodes[0]] - eeg_50s[electrodes[1]]
    
    return transformed_eeg



In [6]:
def montage_transform_transverse_bipolar(eeg_50s):
    montage_pairs= {
    ###Top1
    'F7-Fp1': ('F7', 'Fp1'),  # 将值更正为键中电极的正确配对
    'Fp1-Fp2': ('Fp1', 'Fp2'),
    'Fp2-F8': ('Fp2', 'F8'),
    ####TOp2
    'F7-F3': ('F7', 'F3'),
    'F3-Fz': ('F3', 'Fz'),
    'Fz-F4': ('Fz', 'F4'),
    'F4-F8': ('F4', 'F8'),
    ####Middle
    'T3-C3': ('T3', 'C3'),
    'C3-Cz': ('C3', 'Cz'),
    'Cz-C4': ('Cz', 'C4'),
    'C4-T4': ('C4', 'T4'),
    ###Lower1
    'T5-P3': ('T5', 'P3'),
    'P3-Pz': ('P3', 'Pz'),
    'Pz-P4': ('Pz', 'P4'),
    'P4-T6': ('P4', 'T6'),
    ####Lower2
    'T5-O1': ('T5', 'O1'),
    'O1-O2': ('O1', 'O2'),
    'O2-T6': ('O2', 'T6'),
    
    'EKG': ('EKG',)  # 似乎已正确，无需修改
    }
    
    transformed_eeg = pd.DataFrame()
    
    for new_col, electrodes in montage_pairs.items():
        if len(electrodes) == 1: 
            transformed_eeg[new_col] = eeg_50s[electrodes[0]]
        else:
            transformed_eeg[new_col] = eeg_50s[electrodes[0]] - eeg_50s[electrodes[1]]
    
    return transformed_eeg



In [7]:
def process_eeg_and_save_pickle(df, raw_data_dir, target_cols, output_file):
    """
    save eeg to dic,  {  {id, offset seconds, numpy array, target}, {...}, {...} }。
    
    - df: DataFrame with eeg_id、eeg_label_offset_seconds and target colomns 
    - raw_data_dir: Parquet files path folder。
    - target_cols: target colomns, ['seizure_vote', 'lpd_vote', ...]。
    """
    eeg_data_dict = {}
    count = 0
    for eeg_id in df['eeg_id'].unique():
        eeg_id_str = str(eeg_id)
        eeg_path = f"{raw_data_dir}/train_eegs/{eeg_id_str}.parquet"
        eeg = pd.read_parquet(eeg_path)
        
        offsets = df[df['eeg_id'] == eeg_id]['eeg_label_offset_seconds'].unique()
        
        for offset in offsets:
            eeg_offset = int(offset)
            eeg_50s = eeg.iloc[eeg_offset*200:(eeg_offset+50)*200]
            
            # Montage
            transformed_eeg_np = montage_transform_transverse_bipolar(eeg_50s)
            
            # get target
            target_values = df[(df['eeg_id'] == eeg_id) & (df['eeg_label_offset_seconds'] == offset)][target_cols].values[0]
            
            # save as dix
            eeg_data_dict[f"{eeg_id_str}_{offset}"] = {
                "eeg_id": eeg_id_str,
                "offset": offset,
                "data": transformed_eeg_np,
                "targets": target_values
            }
            count += 1
            if count % 500 == 0:  # Print progress every 500 files
                print(f"Processed {count} files.")
        del eeg
        del transformed_eeg_np
        gc.collect()
    
    with open(output_file, 'wb') as handle:
        pickle.dump(eeg_data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)




In [8]:
TARGET_COLS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
OUT_PUT = os.path.join(PROCESSED_DATA_DIR, "eeg_data_dict_bio_trans.pickle")

In [9]:

process_eeg_and_save_pickle(filtered_df, RAW_DATA_DIR, TARGET_COLS, OUT_PUT)

Processed 500 files.
Processed 1000 files.
Processed 1500 files.
Processed 2000 files.
Processed 2500 files.
Processed 3000 files.
Processed 3500 files.
Processed 4000 files.
Processed 4500 files.
Processed 5000 files.
Processed 5500 files.
Processed 6000 files.
Processed 6500 files.
Processed 7000 files.
Processed 7500 files.
Processed 8000 files.
Processed 8500 files.
Processed 9000 files.
Processed 9500 files.
Processed 10000 files.
Processed 10500 files.
Processed 11000 files.
Processed 11500 files.
Processed 12000 files.
Processed 12500 files.
Processed 13000 files.
Processed 13500 files.
Processed 14000 files.
Processed 14500 files.
Processed 15000 files.
Processed 15500 files.
Processed 16000 files.
Processed 16500 files.
Processed 17000 files.
Processed 17500 files.
Processed 18000 files.
Processed 18500 files.
Processed 19000 files.
Processed 19500 files.
Processed 20000 files.
Processed 20500 files.
Processed 21000 files.
Processed 21500 files.
Processed 22000 files.
Processe